In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from math import radians, sin, cos, asin, sqrt, pi, atan, atan2, fabs
import unicodedata


import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon, Point
from shapely import wkt

In [15]:
pruebas = pd.read_csv("../properati_dataset_modificado_descripciones.csv")
propiedades = pd.read_csv('../../set_datos_sup_desc.csv')

In [4]:
def eliminar_tildes(s):
   return str(''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')))
 

def contar(x, minimo, d):
    lista = x.split()
    for cadena in lista:
        if ',' in cadena:
            cadena = cadena.split(',')
        elif '.' in cadena:
            cadena = cadena.split('.')
        elif ':' in cadena:
            cadena = cadena.split(':')
        elif '-' in cadena:
            cadena = cadena.split('-')
        else:
            cadena = [cadena]
        for palabra in cadena:
            try:
                if len(palabra) < minimo:
                    continue
                palabra = eliminar_tildes(unicode(palabra, "utf-8"))
                palabra = palabra.lower()
                if palabra not in d:
                    d[palabra] = 0
                d[palabra] += 1
            except UnicodeEncodeError:
                continue


In [5]:
d={}
a = pruebas.loc[:, 'description'].apply(lambda x: contar(x, 6,d))

In [6]:
len(d)

28893

In [7]:
for c in d.keys():
    if d[c]< 1000:
        del d[c]

In [8]:
len(d)

107

In [9]:
c = 0
for i in d.keys():
    c+=1
    print(i)
    if c > 10:
        break

traves
calidad
pileta
quincho
categoria
departamentos
central
parque
electrico
inmueble
amplio


In [10]:
l = d.items()
l.sort(key = lambda x: x[1],reverse = True)
l

[('cocina', 12233),
 ('comedor', 9781),
 ('ambientes', 8900),
 ('living', 7435),
 ('balcon', 6364),
 ('completo', 6327),
 ('dormitorios', 6297),
 ('departamento', 6080),
 ('frente', 5789),
 ('dormitorio', 5143),
 ('excelente', 4764),
 ('lavadero', 4637),
 ('edificio', 4623),
 ('amplio', 3897),
 ('planta', 3897),
 ('placard', 3692),
 ('cochera', 3563),
 ('terraza', 3281),
 ('cuenta', 3245),
 ('parrilla', 3133),
 ('cuadras', 3121),
 ('propiedades', 3004),
 ('expensas', 2960),
 ('mesada', 2948),
 ('luminoso', 2724),
 ('acceso', 2698),
 ('entrada', 2568),
 ('medidas', 2524),
 ('ubicacion', 2498),
 ('toilette', 2498),
 ('propiedad', 2474),
 ('ubicado', 2446),
 ('unidades', 2439),
 ('estado', 2356),
 ('cucicba', 2327),
 ('metros', 2203),
 ('banos', 2169),
 ('cocheras', 2165),
 ('servicio', 2136),
 ('corredor', 2093),
 ('principal', 2091),
 ('credito', 2071),
 ('madera', 2034),
 ('salida', 2023),
 ('responsable', 1998),
 ('muebles', 1923),
 ('publicado', 1882),
 ('espacio', 1865),
 ('jardin',

In [11]:
def tiene_cadena(string, lista):
    string = str(string)
    string = string.split()
    for cadena in string:
        if ',' in cadena:
            cadena = cadena.split(',')
        elif '.' in cadena:
            cadena = cadena.split('.')
        elif ':' in cadena:
            cadena = cadena.split(':')
        elif '-' in cadena:
            cadena = cadena.split('-')
        else:
            cadena = [cadena]
        for palabra in cadena:
            try:
                if len(palabra) == 0:
                    continue
                palabra = eliminar_tildes(unicode(palabra, "utf-8"))
                palabra = palabra.lower()
                if palabra in lista:
                    return True
            except UnicodeEncodeError:
                continue
    return False

In [74]:
# Tomamos aquellas palabras que nos parecen importantes
lista = ['cocina', 'comedor', 'living', 'banos', 'balcon','ambientes', 'departamento', 'dormitorios',\
'quincho', 'granito','porcelanato','completo', 'excelente', 'edificio', 'lavadero', 'amplio', 'planta',\
'cuadras', 'placard', 'terraza', 'acceso', 'parrilla', 'luminoso',  'madera', 'credito', 'calidad',\
'cucicba', 'amenities']

In [82]:
cadenas = ['vigilancia', 'seguridad', 'alarma', 'blindado', 'blindaje', 'guardia']
propiedades.loc[:,'seguridad'] = propiedades.loc[:, 'description'].apply(lambda x: tiene_cadena(x, cadenas))
pruebas.loc[:,'seguridad'] = pruebas.loc[:, 'description'].apply(lambda x: tiene_cadena(x, cadenas))

In [83]:
cadenas = ['aire frio', 'acondicionado', 'split', 'acondicionados', 'climatizador', 'central',\
          'radiante', 'pisoradiante', 'radante', 'radiador', 'calefaccion']
propiedades.loc[:,'aire'] = propiedades.loc[:, 'description'].apply(lambda x: tiene_cadena(x, cadenas))
pruebas.loc[:,'aire'] = pruebas.loc[:, 'description'].apply(lambda x: tiene_cadena(x, cadenas))

In [84]:
cadenas = ['pileta', 'piscina', 'picina','pisina', 'yaccuzi', ' jacuzzi', 'jacuzi',\
           'yacuzi', 'yacusi', 'jacussi', 'hidromasaje']
propiedades.loc[:,'pileta'] = propiedades.loc[:, 'description'].apply(lambda x: tiene_cadena(x, cadenas))
pruebas.loc[:,'pileta'] = pruebas.loc[:, 'description'].apply(lambda x: tiene_cadena(x, cadenas))

In [86]:
cadenas = ['cochera', 'cocheras', 'garage', 'estacionamiento']
propiedades.loc[:,'cochera'] = propiedades.loc[:, 'description'].apply(lambda x: tiene_cadena(x, cadenas))
pruebas.loc[:,'cochera'] = pruebas.loc[:, 'description'].apply(lambda x: tiene_cadena(x, cadenas))

In [87]:
cadenas = ['tren', 'subte', 'estacion']
propiedades.loc[:,'transporte'] = propiedades.loc[:, 'description'].apply(lambda x: tiene_cadena(x, cadenas))
pruebas.loc[:,'transporte'] = pruebas.loc[:, 'description'].apply(lambda x: tiene_cadena(x, cadenas))

In [93]:
for palabra in lista:
    propiedades.loc[:,palabra] = propiedades.loc[:, 'description'].apply(lambda x: tiene_cadena(x, [palabra]))
    pruebas.loc[:,palabra] = pruebas.loc[:, 'description'].apply(lambda x: tiene_cadena(x, [palabra]))

In [19]:
# Agrego mas elementos para ver si mejora
lista = ['expensas','mesada','responsable', 'muebles', 'espacio', 'diario', 'independiente', 'cubierta',\
         'contrafrente', 'integrada', 'recepcion', 'categoria', 'profesional', 'baulera', 'natural']
for palabra in lista:
    propiedades.loc[:,palabra] = propiedades.loc[:, 'description'].apply(lambda x: tiene_cadena(x, [palabra]))
    pruebas.loc[:,palabra] = pruebas.loc[:, 'description'].apply(lambda x: tiene_cadena(x, [palabra]))

In [20]:
propiedades.to_csv('../../set_datos_sup_desc.csv', index = False)
pruebas.to_csv('../properati_dataset_modificado_descripciones.csv', index = False)